# What's in an Avocado Toast: A Supply Chain Analysis

You're in London, making an avocado toast, a quick-to-make dish that has soared in popularity on breakfast menus since the 2010s. A simple smashed avocado toast can be made with five ingredients: one ripe avocado, half a lemon, a big pinch of salt flakes, two slices of sourdough bread and a good drizzle of extra virgin olive oil. It's no small feat that most of these ingredients are readily available in grocery stores. 

In this project, you'll conduct a supply chain analysis of three of these ingredients used in an avocado toast, utilizing the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are, fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple categories tags in the `categories_tags` column.
- There is a column in each CSV file called `origins_tags` with strings for country of origin of that item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin, and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

In [21]:
import pandas as pd

# Read in avocado csv into dataframe
avocado = pd.read_csv('data/avocado.csv', sep='\t')

# Dataframe has too many columns to display, showing first 5 rows x 5 columns
avocado.iloc[:5, :5]

,code,lc,product_name_de,product_name_el,product_name_en
0,0059749979702,fr,NaN,NaN,NaN
1,7610095131409,en,NaN,NaN,NaN
2,4005514005578,en,NaN,NaN,Gelbe Linse Avocado Brotaufstrich
3,0879890002513,en,NaN,NaN,Avocado toast chili lime
4,0223086613685,en,NaN,NaN,Avocado


In [22]:
# Create list to subset avocados dataframe columns
subset_cols = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']
# Subset avocado dataframe columns
avocado = avocado[subset_cols]

display(avocado.head(), avocado.columns)

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
0,0059749979702,fr,NaN,NaN,NaN,NaN,Naturalia,naturalia,"en:plant-based-foods-and-beverages,en:plant-ba...",NaN,Canada,en:canada,NaN,NaN
1,7610095131409,en,NaN,NaN,NaN,NaN,Zweifel,zweifel,"en:snacks,en:salty-snacks,en:appetizers,en:chi...","en:vegetarian,en:vegan","Switzerland, World","en:switzerland,en:world",NaN,NaN
2,4005514005578,en,Gelbe Linse Avocado Brotaufstrich,NaN,NaN,NaN,Tartex,tartex,de:abendbrotsufstrich,"en:organic,en:eu-organic,en:eg-oko-verordnung",Germany,en:germany,NaN,NaN
3,0879890002513,en,Avocado toast chili lime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN
4,0223086613685,en,Avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,NaN,NaN


Index(['code', 'lc', 'product_name_en', 'quantity', 'serving_size',
       'packaging_tags', 'brands', 'brands_tags', 'categories_tags',
       'labels_tags', 'countries', 'countries_tags', 'origins',
       'origins_tags'],
      dtype='object')

In [23]:
# Read in relevant_avocado_categories.txt to create a list of relevant categories to search for
with open('data/relevant_avocado_categories.txt', 'r') as file:
    relevant_avocado_cats = file.read().splitlines()
    file.close()
    
relevant_avocado_cats

['en:avocadoes',
 'en:avocados',
 'en:fresh-foods',
 'en:fresh-vegetables',
 'en:fruchte',
 'en:fruits',
 'en:raw-green-avocados',
 'en:tropical-fruits',
 'en:tropische-fruchte',
 'en:vegetables-based-foods',
 'fr:hass-avocados']

In [24]:
# Examine Avocado categories_tags column
display(avocado['categories_tags'].describe(), avocado['categories_tags'].describe().top)

count                                                   752
unique                                                  122
top       en:plant-based-foods-and-beverages,en:plant-ba...
freq                                                    161
Name: categories_tags, dtype: object

'en:plant-based-foods-and-beverages,en:plant-based-foods,en:fruits-and-vegetables-based-foods,en:fruits-based-foods,en:fruits,en:tropical-fruits,en:avocados'

In [25]:
# Transform column to contain list instead of comma seperated str
avocado['categories_list'] = avocado['categories_tags'].str.split(',')
display(avocado['categories_list'].describe(), avocado['categories_list'].describe().top)

count                                                   752
unique                                                  122
top       [en:plant-based-foods-and-beverages, en:plant-...
freq                                                    161
Name: categories_list, dtype: object

['en:plant-based-foods-and-beverages',
 'en:plant-based-foods',
 'en:fruits-and-vegetables-based-foods',
 'en:fruits-based-foods',
 'en:fruits',
 'en:tropical-fruits',
 'en:avocados']

In [26]:
# Drop rows with null values in the categories_list column
avocado = avocado.dropna(subset='categories_list')

avocado['categories_list'].isnull().sum()

0

In [27]:
# Subset avocado dataframe for rows that contain one or more relevant tags in the categories list column
avocado = avocado[avocado['categories_list'].apply(lambda x: any([i for i in x if i in relevant_avocado_cats]))]

avocado['categories_list'].describe()

count                                                   182
unique                                                   18
top       [en:plant-based-foods-and-beverages, en:plant-...
freq                                                    161
Name: categories_list, dtype: object

In [28]:
# Filter avocado for United Kingdom Data
avocado_uk = avocado[avocado['countries'] == 'United Kingdom']
avocado_uk['countries'].value_counts()

countries
United Kingdom    13
Name: count, dtype: int64

In [29]:
# Find most common country for avocado origin
avocado_origin = (avocado_uk['origins_tags'].value_counts().index[0])
avocado_origin = avocado_origin.lstrip("en:")

avocado_origin

'peru'

In [30]:
def read_and_filter_data_by_country(filename, relevant_cats, country):
    # Read ingredient csv into dataframe
    ingredient = pd.read_csv('data/' + filename, sep='\t', low_memory=False)
    
    # Create list to subset ingredient dataframe columns
    subset_cols = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 'labels_tags', 'countries', 'countries_tags', 'origins','origins_tags']
    # Subset ingredient dataframe columns
    ingredient = ingredient[subset_cols]
    
    # Transform column to contain list instead of comma seperated str
    ingredient['categories_list'] = ingredient['categories_tags'].str.split(',')
    
    # Drop rows with null values in the categories_list column
    ingredient = ingredient.dropna(subset='categories_list')
    
    # Subset dataframe for rows that contain one or more relevant tags in the categories list column
    ingredient = ingredient[ingredient['categories_list'].apply(lambda x: any([i for i in x if i in relevant_cats]))]
    
    # Filter for United Kingdom Data
    ingredient_uk = ingredient[ingredient['countries'] == country]
    
    # Find most common country for ingredient origin
    ingredient_origin = (ingredient_uk['origins_tags'].value_counts().index[0])
    ingredient_origin = ingredient_origin.lstrip("en:")
    ingredient_origin = ingredient_origin.replace('-', ' ')
    
    return ingredient_origin

In [31]:
top_avocado_origin = read_and_filter_data_by_country('avocado.csv', relevant_avocado_cats, 'United Kingdom')
top_avocado_origin

'peru'

In [32]:
# Utilize the read_and_filter_data_by_country to determine the top origin country of olive oil for the United Kingdom
with open('data/relevant_olive_oil_categories.txt', 'r', encoding='utf-8') as file:
    relevant_olive_oil_cats = file.read().splitlines()
    file.close()
    
top_olive_oil_origin = read_and_filter_data_by_country('olive_oil.csv', relevant_olive_oil_cats, 'United Kingdom')
top_olive_oil_origin

'greece'

In [33]:
# Utilize the read_and_filter_data_by_country to determine the top origin country of sourdough for the United Kingdom
with open('data/relevant_sourdough_categories.txt', 'r') as file:
    relevant_sourdough_cats = file.read().splitlines()
    file.close()
    
top_sourdough_origin = read_and_filter_data_by_country('sourdough.csv', relevant_sourdough_cats, 'United Kingdom')
top_sourdough_origin

'united kingdom'

In this project, the consistency in formatting across multiple ingredient data frames facilitated the creation and utilization of a user-defined function. This homogeneity across the data frames enabled the seamless analysis of multiple ingredients, providing a streamlined and efficient approach for extracting insights.